In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import scipy.io as sc
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental import preprocessing
import time
from IPython.display import clear_output
import math 

In [5]:
def dataset(dataFile, labelFile):
    dataFile = "../communication/" + dataFile
    labelFile = "../communication/" + labelFile
    my_data = sc.loadmat(dataFile)
    my_labels = sc.loadmat(labelFile)
    my_data = my_data['Y']
    X = my_labels['L_S_x']
    myOrig = table_data(my_data, my_labels['L_Constellations'][0], X)
    mytable = assign_label(myOrig)
    return mytable


def assign_label(data):
    c_4 = [1,-1]
    c_16 = [3,1,-1,-3]
    c_16r = [-3,-1,1,3]
    cons_4 = np.dot(np.sqrt(0.5),[complex(i,j)for i in c_4 for j in c_4])
    cons_16 = np.array([complex(i,j)for j in c_16 for i in c_16r])
    cons_16 = cons_16/np.sqrt(np.mean(np.abs(cons_16)**2))
    cons4 = data[data.cons==1]
    cons4_label = np.array([[cons_4[i-1]]for i in cons4.label])
    cons16 = data[data.cons==2]
    cons16_label = np.array([[cons_16[i-1]]for i in cons16.label.to_numpy().real.astype(int)])
    data[data.cons==2].index
    data['buffer'] = 0
    data['buffer'] = 0
    data.iloc[data[data.cons==1].index, 5] = cons4_label
    data.iloc[data[data.cons==2].index, 5] = cons16_label
    data['label_real'] = data.buffer.to_numpy().real
    data['label_imag'] = data.buffer.to_numpy().imag
    myTest = data.copy()
    myTest.loc[myTest.cons == 2, 'label'] = myTest.loc[myTest.cons == 2, 'label'] + 4
    myTest.label = myTest.label - 1
    return myTest


def table_data(my_data, cons, label):
    block = my_data.shape[1]
    my_data_size = my_data.shape[0] * block
    my_data_div = my_data.T.reshape(my_data_size, )
    cons_array = np.array([[cons[i]] * my_data.shape[0] for i in range(0, block)]).reshape(my_data_size, )
    block_array = np.array([([i + 1] * my_data.shape[0]) for i in range(0, block)]).reshape(my_data_size, )
    label_array = label.T.reshape(my_data_size, )    
    test_pd = pd.DataFrame({'real': my_data_div.real, 'imag': my_data_div.imag,
                            'cons': cons_array, 'block': block_array,
                            'label': label_array})
    return test_pd

In [22]:
data1 = "hard"
data1_label = "hard_label"
data = dataset(data1, data1_label)
test = data.loc[:, ('real', 'imag')]

In [163]:
def make_generator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(50, use_bias=False, input_shape=[50,2]))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(2))
    return model

In [164]:
test_feature = np.array(test)

In [165]:
test_feature.shape

(50, 2)

In [166]:
generator_s = make_generator()
generator_n = make_generator()
generator_i = make_generator()

In [167]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(20, use_bias=False, input_shape=[50,2]))
    model.add(layers.Dense(50, activation = 'sigmoid'))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model
discriminator_t = make_discriminator_model()
discriminator_d = make_discriminator_model()
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)
def identity_loss(real, fake):
    loss = tf.reduce_mean(tf.abs(real - fake))
    return LAMBDA * 0.5 * loss

In [168]:
generator_s_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_n_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_i_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_d_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_t_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [169]:
@tf.function
def train_step(total, label):
    with tf.GradientTape(persistent=True) as tape:
        s = generator_s(total, training=True)
        n = generator_n(total, training=True)
        i = generator_i(total, training=True)
        gen = (s + n + i)
        gen = tf.reshape(gen, (1,50,2))
        fake_t = discriminator_t(gen, training=True)
        real_t = discriminator_t(total, training=True)
        gen_loss = generator_loss(gen)
        fake_d = discriminator_d(s, training=True)
        real_d = discriminator_d(label, training=True)
        disc_t_loss = discriminator_loss(real_t, fake_t)
        disc_d_loss = discriminator_loss(real_d, fake_d)
        identity_s_loss = identity_loss(label, s)
        identity_g_loss = identity_loss(total, gen)
        total_s_loss = 0.2*(gen_loss+identity_g_loss) + 0.8*(identity_s_loss)
        total_n_loss = identity_g_loss + gen_loss
        total_i_loss = identity_g_loss + gen_loss

    gradients_of_s_generator = tape.gradient(total_s_loss, generator_s.trainable_variables)
    gradients_of_i_generator = tape.gradient(total_i_loss, generator_i.trainable_variables)
    gradients_of_n_generator = tape.gradient(total_n_loss, generator_n.trainable_variables)
    gradients_of_discriminator_t = tape.gradient(disc_t_loss, discriminator_t.trainable_variables)
    gradients_of_discriminator_d = tape.gradient(disc_d_loss, discriminator_d.trainable_variables)
    generator_s_optimizer.apply_gradients(zip(gradients_of_s_generator, generator_s.trainable_variables))
    generator_i_optimizer.apply_gradients(zip(gradients_of_i_generator, generator_i.trainable_variables))
    generator_n_optimizer.apply_gradients(zip(gradients_of_n_generator, generator_n.trainable_variables))
    discriminator_t_optimizer.apply_gradients(zip(gradients_of_discriminator_t, discriminator_t.trainable_variables))
    discriminator_d_optimizer.apply_gradients(zip(gradients_of_discriminator_d, discriminator_d.trainable_variables))


In [197]:
EPOCHS = 50
LAMBDA = 10

In [177]:
def shuffle_data(my_table):
    real_y = (2*my_table.real.min())/(my_table.real.max() - my_table.real.min()) + 1
    real_x = (my_table.real.max()) / (1 + real_y)
    imag_y = (2*my_table.imag.min())/(my_table.imag.max() - my_table.imag.min()) + 1
    imag_x = (my_table.imag.max()) / (1 + imag_y)
    my_table.real = (my_table.real / real_x) - real_y
    my_table.imag = (my_table.imag/ imag_x) - imag_y
    train_feature = data.loc[:, ('real', 'imag')]
    train_label = data.loc[:, ('label_real', 'label_imag')]
    test_feature = tf.cast(train_feature, tf.float32)
    test_label = tf.cast(train_label, tf.float32)
    test_feature = tf.reshape(test_feature,(1000,1,50,2))
    test_label = tf.reshape(test_label, (1000,1,50,2))
    symbol = data.loc[:, 'label']
    return test_feature, test_label

In [172]:
f, l = shuffle_data(data)

In [173]:
discriminator_t(tf.reshape(generator_s(f[0]), (1,50,2)))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.20871271]], dtype=float32)>

In [174]:
discriminator_t(l[0])

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.39248267]], dtype=float32)>

In [198]:
for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    for i in range(len(f)):
        test = f[i]
        label = l[i]
        train_step(test, label)
        if n % 10 == 0:
            print ('.', end='')
            n+=1
        
    if  ((epoch+1)%5) == 0:
        print('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                           time.time() - start))
    if  ((epoch+1)%5) == 0:
        id = str(epoch)
        s = generator_s(test_feature, training=False)
        i = generator_i(test_feature, training=False)
        n = generator_n(test_feature, training=False)
        gen = s + i + n
        test = identity_loss(s, test_label)
        gen_loss = identity_loss(gen, test_feature)
        print("_____Test Result:_____")
        print('The generator total loss is', gen_loss)
        print('The signal loss is ', test)
        print("___________________\n")

.....Time taken for epoch 5 is 0.9467928409576416 sec

_____Test Result:_____
The generator total loss is tf.Tensor(0.026924517, shape=(), dtype=float32)
The signal loss is  tf.Tensor(3.7830434, shape=(), dtype=float32)
___________________

.....Time taken for epoch 10 is 0.9386239051818848 sec

_____Test Result:_____
The generator total loss is tf.Tensor(0.017396772, shape=(), dtype=float32)
The signal loss is  tf.Tensor(3.769917, shape=(), dtype=float32)
___________________

.....Time taken for epoch 15 is 0.9241838455200195 sec

_____Test Result:_____
The generator total loss is tf.Tensor(0.016121972, shape=(), dtype=float32)
The signal loss is  tf.Tensor(3.7655833, shape=(), dtype=float32)
___________________

.....Time taken for epoch 20 is 0.9161090850830078 sec

_____Test Result:_____
The generator total loss is tf.Tensor(0.018865064, shape=(), dtype=float32)
The signal loss is  tf.Tensor(3.7654552, shape=(), dtype=float32)
___________________

.....Time taken for epoch 25 is 0.

In [194]:
len(f)

1000

In [193]:
l[0][0][:10]

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[ 0.9486833 , -0.31622776],
       [ 0.31622776, -0.9486833 ],
       [ 0.9486833 ,  0.9486833 ],
       [ 0.31622776,  0.9486833 ],
       [ 0.9486833 ,  0.9486833 ],
       [-0.31622776,  0.31622776],
       [-0.9486833 ,  0.31622776],
       [ 0.31622776, -0.9486833 ],
       [-0.9486833 ,  0.9486833 ],
       [-0.31622776, -0.31622776]], dtype=float32)>